In [1]:
!python -V

Python 3.9.7


In [2]:
import pandas as pd
from numpy import std
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error

In [3]:
df = pd.read_parquet('./data/yellow_tripdata_2022-01.parquet')
len(df.columns)

19

In [4]:
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
std(df.duration)

46.44529571272228

In [5]:
len(df)

2463931

In [6]:
df1 = df[(df.duration >= 1) & (df.duration <= 60)]
len(df1)

2421440

In [7]:
len(df1)/len(df)

0.9827547930522406

In [8]:
# define categorical and numerical columns we will use
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

df1[categorical] = df1[categorical].astype(str)
# transform the DataFrame to a dictionary because is what DictVectorizer() expects
dicts = df1[categorical + numerical].to_dict(orient='records')

# this class does OHE directly without needing us to implement it
dv = DictVectorizer()
# we get the training matrix
X = dv.fit_transform(dicts)
X

/opt/miniconda3/envs/mlops-course/lib/python3.9/site-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


<2421440x516 sparse matrix of type '<class 'numpy.float64'>'
	with 7264320 stored elements in Compressed Sparse Row format>

## Training

In [9]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    # we want to predict the duration so we calculate it in minuts
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    # filter out outliers of less than 1min and more than 60min
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    # transform categorical columns to string
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [10]:
df_train = read_dataframe('./data/yellow_tripdata_2022-01.parquet')
df_val = read_dataframe('./data/yellow_tripdata_2022-02.parquet')

In [11]:
len(df_train), len(df_val)

(2421440, 2918187)

In [12]:
# define categorical and numerical columns we will use
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

# transform the DataFrame to a dictionary because is what DictVectoriizer() expects
train_dicts = df_train[categorical + numerical].to_dict(orient='records')

# this class does OHE directly without needing us to implement it
dv = DictVectorizer()
# we get the training matrix
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

# define the column we want to predict and select the data
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

# train the model
lr = LinearRegression()
lr.fit(X_train, y_train)

# predict for training data
y_pred = lr.predict(X_train)
mean_squared_error(y_train, y_pred, squared=False)

6.986326969515353

In [13]:
# predict for validation data
y_pred = lr.predict(X_val)

# let's see the error we get with this first iteration (in minuts)
mean_squared_error(y_val, y_pred, squared=False)

7.786091896902708